## 12. RDD

- 스파크 저수준 API는 RDD, SparkContext 그리고 어큐뮬레이터와 브로드캐스트 변수 같은 분산형 공유 변수등을 의미함.

### 12.1 저수준 API란

- 분산 데이터 처리를 위한 RDD와 브로드캐스트 변수와 어큐뮬레이터처럼 분산형 공유 변수를 배포하고 다루기 위한 API

#### 12.1.1 저수준 API는 언제 사용할까 

- 고수준 API에서 제공하지 않는 기능이 필요할 경우 (ex: 클러스터의 물리적 데이터의 배치를 아주 세밀하게 제어해야되는 상황)
- RDD를 사용해 개발된 기존 코드를 유지해야되는 경우
- 사용자가 정의한 공유 변수를 다뤄야 하는 경우

DataFrame 트랜스포메이션을 호출하면 실제로 다수의 RDD 트렌스포메이션으로 변환된다.
-> 복잡해지는 워크로드를 디버깅하는 작업이 더욱 쉬워질 수 있음

#### 12.1.2 저수준 API는 어떻게 사용할까
- SparkContext는 저수준 API 기능을 사용하기 위한 진입 지점.
- 스파크 클러스터에서 연산을 수행하는 데 필요한 도구인 SparkSession을 이용해 SparkContext에 접근할 수 있음

In [1]:
spark.sparkContext

<SparkContext master=local[*] appName=PySparkShell>

### 12.2 RDD 개요

- RDD는 스파크 1.x 버전의 핵심 API
- 2.x 버전부터 잘 사용하지 않고 있음 (3.x 버전도 마찬가지)
- 그러나 사용자가 실행한 모든 DataFrame이나 DataSet 코드는 RDD로 컴파일 되고, 스파크 UI에서 RDD 단위로 잡이 수행되고 있음. 

- RDD는 불변성을 가지고 병렬로 처리할 수 있는 파티셔닝된 레코드의 모음.
 - DataFrame의 각 레코드는 스키마를 알고 있는 필드로 구성된 구조화된 로우
 - RDD의 레코드는 그저 프로그래머가 선택하는 자바, 스칼라, 파이썬의 객체 -> 완벽하게 제어할 수 있음 
 <br>
   -> 사용자가 원하는 포맷을 사용해 원하는 모든 데이터를 저장할 수 있음
- RDD는 모든 값을 다루거나, 값 사이의 상호작용 과정을 반드시 수동으로 정의해야함. 
- 구조적 API와 다르게 레코드의 내부 구조를 스파크에서 파악할 수 없기 때문에 최적화 과정에서 많은 수작업을 거쳐야함
 - 구조적 API에 구현된 모든 것들을 수작업으로 다 구현해야된다고 이해하면 됨. 

#### 12.2.1 RDD 유형

- RDD에는 수많은 하위 클래스 있음 
- RDD는 DataFrame API에서 최적화된 물리적 실행 계획을 만드는 데 대부분 사용됨
- 두가지 RDD 타입
 - 제네릭 RDD
 - 키 기반의 집계가 가능한 키-값 RDD
- 둘 다 컬렉션을 표현하지만 키-값 RDD는 특수 연산뿐만 아니라 키를 이용한 사용자 지정 파티셔닝 개념을 가지고 있음

- RDD의 주요 속성
 1. 파티션의 목록
 2. 각 조각을 연산하는 함수
 3. 다른 RDD와의 의존성 목록
 4. 부가적으로 키-값 RDD를 위한 Partitioner(ex: RDD는 해시 파티셔닝이 되어 있음)
 5. 부가적으로 각 조각을 연산하기 위한 기본 위치 목록(ex: 하둡 분산 파일 시스템 파일의 블록 위치)
 
- RDD는 분산 환경에서 데이터를 다루는 데 필요한 지연 처리 방식의 트랜스포메이션과 즉시 실행 방식의 액션을 제공함. 
- RDD에는 로우라는 개념이 없고 객체이기 때문에 구조적 API가 제공하는 함수를 사용할 수 없음 -> 수동으로 처리 
- 원형객체를 파이썬으로 RDD를 다루면 상당한 성능 저하 발생 
 - 직렬화 과정을 거친 데이터를 파이썬 프로세스에 전달하고, 파이썬에서 처리가 끝나면 다시 직렬화하여 자바 가상 머신에 반환하기 때문에 높은 오버헤드 발생 (사용자 정의 함수를 각 로우에 적용하기 때문)

#### 12.2.2 RDD는 언제 사용할까
- 물리적으로 분산된 데이터(자체적으로 구성한 데이터 파티셔닝)에 세부적인 제어가 필요할 때 RDD를 사용해야함

#### 12.2.3 DataSet과 RDD의 케이스 클래스
- DataSet은 구조적 API가 제공하는 풍부한 기능과 최적화 기법을 제공함
- DataSet을 사용하면 JVM 데이터 타입과 스파크 데이터 타입 중 어떤 것을 쓸지 고민하지 않아도 됨

### 12.3 RDD 생성하기 

#### 12.3.1 DataFrame, Dataset으로 RDD 생성

In [2]:
# 파이썬은 DataFrame만 존재하기 때문에 Row타입의 RDD 형성
spark.range(10).rdd

MapPartitionsRDD[5] at javaToPython at NativeMethodAccessorImpl.java:0

In [3]:
spark.range(10).rdd.toDF().show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [4]:
# rdd 메서드는 row 타입을 가진 RDD를 생성함. 
# Row 타입은 스파크가 구조적 API에서 데이터를 표현하는 데 사용하는 내부 카탈리스트 포맷 -> 구조적, 저수준 API 사이를 오고가게 만들 수 있음

#### 12.3.2 로컬 컬렉션으로 RDD 생성하기
- 컬랙션 객체를 RDD로 만들려면 SparkContext의 Parallelize 메서드 필요.
 - 단일 노드 안에 있는 컬렉션을 병렬 컬렉션으로 전환

In [5]:
myCollection = 'Spark The Definitive Guide: Big Data Processing Made Simple'.split(' ')
words = spark.sparkContext.parallelize(myCollection, 2)
words.collect()

['Spark',
 'The',
 'Definitive',
 'Guide:',
 'Big',
 'Data',
 'Processing',
 'Made',
 'Simple']

In [6]:
# 이름을 지정하면 스파크 UI에 지정한 이름으로 RDD 표시
words.setName('myWords')
words.name()

'myWords'

#### 12.3.3 데이터소스로 RDD 생성하기 
- DataSource API를 사용해서 RDD를 생성하는 게 좋음 
- RDD는 주로 RDD 간의 의존성 구조와 파티션 목록을 정의함 -> SparkContext를 사용해서 데이터를 RDD로 읽을 수 있음 
<code>spark.sparkContext.wholeTextFiles('some/file/path/text.txt')</code>
- 생성된 RDD에서 파일명은 첫 번째 객체인 RDD의 키가 되고, 텍스트 파일의 값은 두 번째 문자열 객체인 RDD의 값이 됨 

### 12.4 RDD 다루기
- RDD는 DataFrame 다루는 것과 비슷함
- RDD는 스파크 데이터 타입 대신 자바나 스칼라의 객체를 다루는 것이 가장 큰 차이
- 필터, 맵 함수, 집계 그리고 DataFrame의 다양한 함수를 직접 정의해야함 

### 12.5 트랜스포메이션
- RDD 트랜스포메이션은 구조적 API에서 사용할 수 있는 기능을 가지고 있음
- DataFrame이나 DataSet에 동일하게 RDD에 트랜스포메이션을 지정해 새로운 RDD 생성 가능 

#### 12.5.1 distinct

In [7]:
# 중복제거
print(words.count())
print(words.distinct().count())

9
9


#### 12.5.2 filter
- sql의 where 조건절과 비슷함 
- RDD의 레코드를 모두 확인하고 조건 함수를 만족하는 레코드만 반환 
- 필터 함수로 동작하기 때문에 불리언 타입을 반환해야함 -> 모든 로우는 어떤 경우라도 입력값을 가지고 있어야함

In [8]:
# 함수 정의 S가 시작으로 있는 단어만 뽑기
def startsWithS(individual):
    return individual.startswith("S")

In [9]:
words.filter(lambda word: startsWithS(word)).collect()

['Spark', 'Simple']

### 12.5.3 map
- 주어진 입력을 원하는 값으로 반호나하는 함수를 명시하고 레코드별로 적용

In [10]:
words2 = words.map(lambda word: (word, word[0], word.startswith('S')))

In [11]:
# 불리언 값으로 필터링
words2.filter(lambda record: record[2]).take(5)

[('Spark', 'S', True), ('Simple', 'S', True)]

In [12]:
# flatmap: map 함수의 확장 버전
# 단일 로우를 여러 로우로 변환해야되는 경우가 있음
# 반복처리할 수 있는 형태임

words.flatMap(lambda word: list(word)).take(5)

['S', 'p', 'a', 'r', 'k']

#### 12.5.4 sortBy

In [13]:
words.sortBy(lambda word: len(word)*-1).take(2)

['Definitive', 'Processing']

#### 12.5.5 randomsplit
- RDD를 임의로 분할해 RDD 배열을 만들 때 사용, 가중치와 난수 시드로 구성된 배열을 파라미터로 사용

In [14]:
fiftyFiftySplit = words.randomSplit([0.5,0.5])
print(fiftyFiftySplit)
print(fiftyFiftySplit[0].collect())
print(fiftyFiftySplit[1].collect())

[PythonRDD[39] at RDD at PythonRDD.scala:53, PythonRDD[40] at RDD at PythonRDD.scala:53]
['Spark', 'The', 'Definitive', 'Guide:', 'Big', 'Processing', 'Simple']
['Data', 'Made']


### 12.6 액션
- 지정된 트랜스포메이션 연산을 시작하려면 액션을 사용함
- 액션은 데이터를 드라이버로 모으거나 외부 데이터소스로 내보낼 수 있음

#### 12.6.1 reduce
- RDD의 모든 값을 하나로 만들기 위해선 reduce 매서드를 사용해야함 

In [15]:
spark.sparkContext.parallelize(range(1,21)).reduce(lambda x, y: x+y)

210

In [16]:
# 단어 집합에서 가장 긴 단어를 찾는 예제
# 두개의 결과를 하나로 합쳐주기 때문에 reduce 쓸 수 있음
# 파티션에 대한 리듀스 연산은 비결정적 특성을 가짐 
# 같은 길이일 경우 둘 중 하나가 나올 수 있다. 

def wordLengthReducer(leftWord, rightWord):
    if len(leftWord) > len(rightWord):
        return leftWord
    else:
        return rightWord
    
words.reduce(wordLengthReducer)

'Processing'

#### 12.6.2 count
- <code>words.count()</code>를 사용하면 간략하게 개수를 구할 수 있다. 
- countApprox는 count 함수의 근사치를 제한 시간 내에 계산함 -> 제한시간 초과 시 불완전한 결과 반환
- confidence는 실제 연산한 결과와의 오차율을 의미 

In [17]:
# 특정값을 confidence로 설정하면 실제 연산 결과와 동일한 값이 특정값 이상 포함될 것으로 기대할 수 있음
# 신뢰도 [0,1]사이이고, 벗어나면 예외 발생
words.countApprox(400, 0.95)

9

- countApproxDistinct 메서드의 첫 번째 구현체에서는 상대 정확도를 파라미터로 사용함
- 값을 작게 하면 더 많은 메모리 공간을 사용하는 카운터가 생성됨
- 설정값은 0.000017보다 커야함


In [18]:
words.countApproxDistinct(0.05)

9

- 다른 구현체 사용시 동작을 세부적으로 제어할 수 있음.
- 상대 정확도를 지정할 때 두 개의 파라미터를 사용함
 - 하나는 '일반' 데이터를 위한 파라미터, 하나는 희소 표현을 위한 파라미터

- 두개의 인수 p와 sp는 정밀도와 희소 정밀도
 - 카디널리티가 작을 때 0이 아닌 값 (sp > p)를 설정하면 메모리 소비를 줄이면서 정확도를 증가시킬 수 있음 
 

In [19]:
# 왜 안되죠?
words.countApproxDistinct(4,10)

TypeError: countApproxDistinct() takes from 1 to 2 positional arguments but 3 were given

- countByValue는 RDD 값의 개수를 구할 수 있음
- 결과 데이터셋을 드라이버의 메모리로 읽어들여 처리함
- 익스큐터 연산 결과가 드라이버 메모리에 모두 적제됨 -> 결과가 작은 경우에만 사용

In [20]:
words.countByValue()

defaultdict(int,
            {'Spark': 1,
             'The': 1,
             'Definitive': 1,
             'Guide:': 1,
             'Big': 1,
             'Data': 1,
             'Processing': 1,
             'Made': 1,
             'Simple': 1})

- countByValueApprox는 count와 동일한 연산을 수행하지만 근사치를 계산함 -> 제한시간 내에 처리
- 제한시간을 초과하면 불완전한 결과를 반환 (파이썬엔 이 모듈이 없는 거 같음)

#### 12.6.3 first

In [21]:
# 가장 첫번째 반환
words.first()

'Spark'

#### 12.6.4 max와 min


In [22]:
# 가장 큰값과 작은 값 반환
print(spark.sparkContext.parallelize(range(1,20)).max())
print(spark.sparkContext.parallelize(range(1,20)).min())

19
1


#### 12.6.5 take
- take와 take의 파생 매서드는 RDD에서 가져올 값의 개수를 파라미터로 사용
- takeOrdered, takeSample, top과 같은 유사 함수 있음
- RDD에서 고정 크기의 임의 표본 데이터를 얻기 위해 takeSample 함수를 사용할 수 있음
- takeSample은 withReplacement, 임의 표본 수, 난수 시드값을 파라미터로 사용함 
- top은 암시적 순서에 따라 최상위값을 선택하기 때문에 takeOrdered와는 반대 개념

In [23]:
print(words.take(5))
print(words.takeOrdered(5))
print(words.top(5))
print(words.takeSample(True, 6, 1000))

['Spark', 'The', 'Definitive', 'Guide:', 'Big']
['Big', 'Data', 'Definitive', 'Guide:', 'Made']
['The', 'Spark', 'Simple', 'Processing', 'Made']
['Data', 'Definitive', 'Data', 'Spark', 'Definitive', 'Simple']


### 12.7 파일 저장하기
- RDD 사용하면 일반적인 의미의 데이터소스에 '저장'을 할 수 없음
- 각 파티션 내용을 저장하려면 전체 파티션을 순회하면서 외부 데이터베이스에 저장해야함
- 고수준 API의 내부 처리 과정을 저수준 API로 구현하는 접근법

#### 12.7.1 saveAsTextFile
- 필요한 경우 압축 코덱을 설정할 수 있음

In [24]:
words.saveAsTextFile('file:/tmp/booktitle')

- 하둡에서 사용 가능한 코덱을 임포트 하자 
- 스칼라에만 있는 거 같아서 패스
- <code>words.saveAsTextFile('file:/tmp/booktitle', classof[BZip2Codex])</code>

#### 12.7.2 시퀀스 파일
- 스파크는 다양한 하둡 기능과 호환됨
- 시퀀스 파일은 바이너리 키-값으로 구성된 플랫 파일
- 맵리듀스의 입출력 포맷으로 사용됨
- <code> words.saveAsObjectFile('file:/tmp/booktitle')</code>

#### 12.7.3 하둡 파일
- 클래스, 출력 포맷, 하둡 설정 그리고 압축방식을 지정할 수 있음. 
- 하둡 에코시스텀이나 기존의 맵리듀스 잡을 깊이 있게 다루는 경우가 아니라면 크게 관련 없음 

### 12.8 캐싱
- RDD 캐싱에도 DataFrame이나 DataSet의 캐싱과 동일한 원칙이 적용됨
- 기본적으로 캐시와 저장은 메모리에 있는 데이터만을 대상으로 함

In [27]:
words.cache()

myWords ParallelCollectionRDD[21] at parallelize at PythonRDD.scala:195

In [28]:
# 저장소 수준 조회 -> 20장에서 자세하게 다룸
words.getStorageLevel()

StorageLevel(False, True, False, False, 1)

### 12.9 체크포인팅
- RDD를 디스크에 저장하는 방식
- 나중에 저장된 RDD를 참조할 때는 원본 데이터소스를 다시 계산해 RDD를 생성하지 않고 디스크에 저장된 중간 결과 파티션을 참조
- 메모리에 저장하지 않고 디스크에 저장하는 것 외에는 캐싱과 비슷 
- 반복적인 연산 수행 시 유용
- <code>spark.sparkContext.setCheckpointdir('/some/path/for/checkpointing')</code>
- <code>words.checkpoint()</code>
- 체크포인트에 저장된 RDD를 사용하기 때문에 최적화에 유용함

### 12.10 RDD 시스템 명령으로 전송하기 
- pipe 메서드를 사용하면 파이핑 요소로 생성된 RDD를 외부 프로세스로 전달할 수 있음
- 외부 프로세스는 한 번씩 처리해 결과 RDD를 생성함 
- 입력 파티션은 표준 입력에 전달되고, 결과 파티션은 표준 출력으로 생성됨 


In [32]:
# 파티션 당 5개의 로우를 얻게된다... 정확한 의미를 모르겠음
words.pipe('wc -l').collect()

['4', '5']

#### 12.10.1 mapPartitions
- 스파크는 실제 코드를 실행할 때 파티션 단위로 동작함. 
- map 함수에서 반환하는 RDD의 진짜 형태가 MapPartitionsRDD임
- map은 mapPartitions의 로우 단위 처리를 위한 별칭
- 개별 파티션에 대해 map 연산을 수행할 수 있음 -> 클러스터에서 물리적인 단위로 개별 파티션을 처리하기 때문

In [34]:
words.mapPartitions(lambda part: [1]).sum()

2

- 파티션 단위로 작업을 수행함
- 전체 파티션에 대한 연산을 수행할 수 있음 
- RDD의 전체 하위 데이터셋에 원하는 연산을 수행할 수 있음
- 파티션 그룹의 전체 값을 단일 파티션으로 모은 다음 임의의 함수를 적용하고 제어할 수 있음 
- mapPArtitionsWithIndex 함수를 사용하려면 인덱스(파티션 범위의 인덱스)와 파티션의 모든 아이템을 순회하는 이터레이터를 가진 함수를 인수로 지정해야함. 
- 파티션 인덱스는 RDD의 파티션 번호 -> 레코드가 속한 데이터셋이 어디에 있는지 파악가능, 디버깅 활용 가능 

In [35]:
def indexedFunc(partitionIndex, withinPartIterator):
    return ['partition {} => {}'.format(partitionIndex, x) for x in withinPartIterator]
words.mapPartitionsWithIndex(indexedFunc).collect()

['partition 0 => Spark',
 'partition 0 => The',
 'partition 0 => Definitive',
 'partition 0 => Guide:',
 'partition 1 => Big',
 'partition 1 => Data',
 'partition 1 => Processing',
 'partition 1 => Made',
 'partition 1 => Simple']

#### 12.10.2 foreachPartition
- 파티션의 모든 데이터를 순회할 뿐 결과를 반환하지 않음.
- mapPartitions와 forreachPartition의 차이는 결과값 반환 여부
- 각 파티션의 데이터를 데이터베이스에 저장하는 것과 같이 개별 파티션에서 특정 작업을 수행하는 데 적합한 함수 

#### 12.10.3 glom
- 데이터셋의 모든 파티션을 배열로 변환하는 함수 
- 데이터를 드라이버로 모으고 데이터가 존재하는 파티션의 배열이 필요한 경우에 유용함 
- 파티션이 크거나 파티션 수가 많다면 드라이버가 비정상적으로 종료될 수 있기 때문에 안정성에 문제가 생김

In [36]:
spark.sparkContext.parallelize(['Hello', 'Wolrd'], 2).glom().collect()

[['Hello'], ['Wolrd']]